# Stock Market News and Sentiment Analysis Backtesting

In [54]:
# Libraries
from eod import EodHistoricalData
import pandas as pd
from datetime import datetime, timedelta
from tqdm import tqdm
import nltk
from textblob import TextBlob


# Importing and assigning the api key
with open("../../eodHistoricalData-API.txt", "r") as f:
    api_key = f.read()
    
# EOD Historical Data client
client = EodHistoricalData(api_key)

## Retrieving News

In [2]:
def getNews(ticker, days):
    """
    Retrieves financial news over the course of a specified number of days for
    a given stock ticker.
    """

    # List of news
    news = []
    
    # How many days back to retrieve
    ago = datetime.now() - timedelta(days=days)

    # Getting news over the course of a year
    for i in tqdm(range(10, days, 10)):

        # Grabbing the news
        resp = client.get_financial_news(
            s=ticker,
            from_=(ago+timedelta(days=i-10)).strftime("%Y-%m-%d"),
            to=(ago+timedelta(days=i)).strftime("%Y-%m-%d"),
            limit=50
        )

        # Adding to the news list
        news.extend(resp)
        
    # Filtering out irrelevant news
    lst = [i for i in news if sum(x in {ticker.lower()} for x in nltk.word_tokenize(i['title'].lower()))>=1]
    
    return pd.DataFrame(lst)

In [3]:
# Retrieving financial news
news = getNews("AMD", 365)

100%|██████████| 36/36 [00:21<00:00,  1.66it/s]


## Retrieving Price History

In [4]:
# Date
ago = datetime.now() - timedelta(days=365)

# Prices
prices = pd.DataFrame(client.get_prices_eod("AMD", from_=ago.strftime("%Y-%m-%d")))

In [5]:
prices

,date,open,high,low,close,adjusted_close,volume
0,2020-11-30,87.33,92.7400,86.5300,92.66,92.66,84482979
1,2020-12-01,92.25,93.9000,90.7800,92.63,92.63,58670527
2,2020-12-02,92.89,96.3700,92.5253,93.74,93.74,57988822
3,2020-12-03,94.06,94.7000,92.0100,92.31,92.31,35859716
4,2020-12-04,92.58,94.5800,90.6300,94.04,94.04,45570273
...,...,...,...,...,...,...,...
246,2021-11-19,155.76,156.9200,153.4450,155.41,155.41,41668862
247,2021-11-22,157.14,161.8799,152.3900,152.52,152.52,58674078
248,2021-11-23,150.41,152.6600,145.2996,149.92,149.92,62852705
249,2021-11-24,149.46,157.9300,147.1900,157.80,157.80,60677062


## Sentiment Analysis

In [65]:
news['sentiment'] = news['title'].apply(lambda x: TextBlob(x.lower()).sentiment[0])

In [66]:
news[['sentiment', 'title']].tail(50)

,sentiment,title
350,0.200000,Advanced Micro Devices (AMD) Reports Next Week...
351,0.150000,Advanced Micro Devices (AMD) Stock Sinks As Ma...
352,0.000000,AMD Ryzen™ Threadripper™ PRO Processors Select...
353,0.000000,AMD Ryzen™ Threadripper™ PRO Processors Select...
354,0.400000,Advanced Micro Devices (AMD) to Post Q3 Earnin...
355,0.000000,AMD: What’s on the Menu for Q3 Earnings
356,0.400000,Why Advanced Micro Devices (AMD) Might Surpris...
357,0.000000,"AMD Beats on Q3 Earnings &amp; Revenues, Raise..."
358,0.433333,My Take on AMD's Strong Earnings Report
359,0.000000,AMD Promotes Server SoC Architect Kevin Lepak ...
